In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 13.9 MB/s eta 0:00:00


In [ ]:
from transformers import OpenAIGPTTokenizer, OpenAIGPTModel
import torch
from transformers import GPT2Model
from transformers import GPT2Tokenizer
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup
import numpy as np
import transformers as ppb

In [ ]:
import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS =nltk.corpus.stopwords.words('english')

In [ ]:
def clean_text(text):
    if type(text)!=str:
        return ""
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [ ]:
from transformers import OpenAIGPTTokenizer, OpenAIGPTModel
import torch

In [ ]:
model = GPT2Model.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
def gpt_fscore(text,model,tokenizer):
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)
    outputs = model(input_ids)
    last_hidden_states = outputs[0]
    return last_hidden_states

In [ ]:
fname = '/content/gdrive/My Drive/GitterCom_Project/Codes/GitterCom.csv'
df = pd.read_csv(fname,encoding='utf8')
df.drop([581, 2102, 4858, 6417], axis=0, inplace=True)

non_string = []
for i, row in df.iterrows():
    res = ((type(row['message']) != str) or (type(row['Purpose']) != str) or (type(row['Category']) != str) or (type(row['Subcategory']) != str))
    if res:
        non_string.append(i)
df.drop(non_string, axis=0, inplace=True)
df['message'] = df['message'].apply(clean_text)

train_tokens = list(map(lambda t:   tokenizer.tokenize(t)+['[CLS]'], df['message']))
train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids,train_tokens))
to_be_removed=[]

for i in range(0,len(train_tokens_ids)):
    if len(train_tokens_ids[i])>512:
        to_be_removed.append(i)
        print(len(to_be_removed))
df.drop(to_be_removed,axis=0,inplace=True)
removed=0

for i in to_be_removed:
    train_tokens_ids.pop(i-removed)
    removed=removed+1
print(to_be_removed)


b=np.zeros((np.shape(df)[0]))
for i in range(0,np.shape(df)[0]):
    b[i]=np.shape(train_tokens_ids[i])[0]
pad=np.zeros((np.shape(df)[0],int(np.max(b))))

for i in range(0,np.shape(df)[0]):
    a=train_tokens_ids[i]
    for j in range(0,np.shape(a)[0]):
        pad[i,j]=a[j]

[]


In [ ]:
pad.shape

(9959, 319)

In [ ]:
output = []
x = 36
j = x

for i in range(j, j+4):
  print(i)
  if i == 39:
    pad1 = pad[i*250:,:]
  else:
    pad1 = pad[i*250:(i+1)*250,:]
  input_ids = torch.tensor(np.array(pad1))
  print(input_ids.shape)
  with torch.no_grad():
    last_hidden_states = model(input_ids.long())
  features = last_hidden_states[0][:,0,:].numpy()
  print(features.shape)
  output.append(features)

len(output)

36
torch.Size([250, 319])
(250, 768)
37
torch.Size([250, 319])
(250, 768)
38
torch.Size([250, 319])
(250, 768)
39
torch.Size([209, 319])
(209, 768)


4

In [ ]:
for i in range(1, 4):
  output[0] = np.vstack((output[0],output[i]))
output[0].shape

(959, 768)

In [ ]:
fname = '9_' + str(int(x/4)) + '.csv'
np.savetxt(fname, output[0], delimiter=',', fmt='%f')

In [ ]:
fname='/content/gdrive/My Drive/GitterCom_Project/Codes/9_0.csv'
data=np.loadtxt(fname,delimiter=',')

In [ ]:
for i in range(1, 10):
  fname1 = '/content/gdrive/My Drive/GitterCom_Project/Codes/9_' + str(i) + '.csv'
  data1 = np.loadtxt(fname1, delimiter=',')
  data = np.vstack((data, data1))

In [ ]:
fname = '9' + '.csv'
np.savetxt(fname, data, delimiter=',', fmt='%f')